In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
from ipywidgets import IntSlider, FloatSlider, VBox, interact


In [ ]:
class Node:
    def __init__(self, id, threshold=0.5):
        self.id = id
        self.state = 0  # 0 = inactive, 1 = active
        self.threshold = threshold
        self.input_signal = 0
        self.refractory_timer = 0

    def update_state(self):
        if self.refractory_timer > 0:
            self.state = 0
            self.refractory_timer -= 1
        elif self.input_signal > self.threshold:
            self.state = 1
            self.refractory_timer = 3
        else:
            self.state = 0
        self.input_signal = 0

class DynamicNetwork:
    def __init__(self, num_nodes):
        self.graph = nx.Graph()
        for i in range(num_nodes):
            self.graph.add_node(i, node=Node(i, threshold=np.random.uniform(0.3, 0.7)))
        for _ in range(num_nodes * 2):
            self.graph.add_edge(
                np.random.randint(0, num_nodes),
                np.random.randint(0, num_nodes),
                weight=np.random.uniform(0.1, 1.0)
            )
        self.stimulus_node = np.random.randint(0, num_nodes)

    def propagate_signal(self, stimulus_prob=0.1):
        if np.random.rand() < stimulus_prob:
            self.graph.nodes[self.stimulus_node]['node'].state = 1
        
        for edge in self.graph.edges(data=True):
            source, target = edge[0], edge[1]
            signal = self.graph.nodes[source]['node'].state * edge[2]['weight']
            self.graph.nodes[target]['node'].input_signal += signal

        for node_id in self.graph.nodes:
            self.graph.nodes[node_id]['node'].update_state()

    def get_active_nodes(self):
        return [n for n, d in self.graph.nodes(data=True) if d['node'].state == 1]

In [ ]:
# Initialize network with adjustable sliders for parameters
def initialize_network():
    num_nodes_slider = IntSlider(value=10, min=5, max=50, step=1, description='Nodes')
    stimulus_prob_slider = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='Stimulus Prob')
    
    def create_network(num_nodes, stimulus_prob):
        network = DynamicNetwork(num_nodes)
        return network, stimulus_prob
    
    return interact(create_network, num_nodes=num_nodes_slider, stimulus_prob=stimulus_prob_slider)

# Interactive setup for the network
network, stimulus_prob = initialize_network()

In [ ]:
def draw_network(graph, active_nodes, ax, pos):
    ax.clear()
    nx.draw(
        graph, pos, ax=ax, with_labels=True,
        node_color=['red' if n in active_nodes else 'blue' for n in graph.nodes],
        edge_color='gray', node_size=500, font_size=8
    )
    ax.set_title('Dynamic Neural Network')

In [ ]:
def animate_network(network, steps=50, save_file=None):
    fig, ax = plt.subplots(figsize=(6, 6))
    pos = nx.spring_layout(network.graph)
    
    def update(frame):
        network.propagate_signal(stimulus_prob)
        active_nodes = network.get_active_nodes()
        draw_network(network.graph, active_nodes, ax, pos)
        ax.set_title(f'Step {frame + 1}')
    
    anim = FuncAnimation(fig, update, frames=steps, repeat=False)
    if save_file:
        anim.save(save_file, writer='ffmpeg', fps=10)
    else:
        display(HTML(anim.to_jshtml()))
    
    return anim

# Run animation and save it
animation = animate_network(network, steps=100, save_file='network_simulation.mp4')